# Install Required Packages

In order to run this notebook, you need to have several Python packages installed. The commands below will install the following packages:

- **datasets:** Provides access to a wide range of datasets and tools to load and process them.
- **transformers:** Contains pre-trained models and tools for working with transformer-based architectures.
- **evaluate:** Offers utilities to compute evaluation metrics.
- **numpy:** A fundamental package for numerical computing in Python.
- **codecarbon:** Tracks energy consumption and emissions during model training.



In [2]:
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install numpy
!pip install codecarbon

# Import Necessary Libraries

This cell imports all the required libraries for loading datasets, preprocessing, model training, evaluation, and energy consumption tracking. Here's what each import does:

- **load_dataset (from datasets):**  
  - Loads datasets from Hugging Face's Datasets library. It supports a variety of datasets for natural language processing (NLP) tasks.

- **AutoTokenizer, AutoModelForSequenceClassification (from transformers):**  
  - `AutoTokenizer` automatically loads the appropriate tokenizer for a given model.
  - `AutoModelForSequenceClassification` loads a pretrained transformer model designed for text classification.

- **TrainingArguments, Trainer (from transformers):**  
  - `TrainingArguments` allows you to configure parameters like batch size, learning rate, and number of epochs.
  - `Trainer` handles the training and evaluation processes using the Hugging Face Transformers library.

- **evaluate:**  
  - Provides tools for computing evaluation metrics such as accuracy, F1-score, or recall.

- **numpy:**  
  - Used for efficient numerical operations and matrix manipulations.

- **DataCollatorWithPadding (from transformers):**  
  - Automatically pads sequences to the maximum length in a batch, ensuring consistent input size for the model.

- **EmissionsTracker (from codecarbon):**  
  - Tracks the carbon footprint of the training process, providing insights into energy consumption and CO₂ emissions.

These imports are essential for loading data, building models, training with transformers, evaluating results, and tracking environmental impact.



In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
from transformers import DataCollatorWithPadding
from codecarbon import EmissionsTracker

# Load the Dataset

This line of code loads the **Phishing Site Classification** dataset using the Hugging Face Datasets library.

***About the Dataset:***

- **Dataset Name:** Phishing Site Classification
- **Source:** Hugging Face (Published by Shawhin)
- **Task:** Binary Classification — Classify URLs as either Safe or Not Safe.
- **Objective:** The dataset is specifically designed to train and evaluate machine learning models for detecting phishing sites.

In [4]:
dataset_dict = load_dataset("shawhin/phishing-site-classification")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## View the Loaded Dataset




In [5]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 2100
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
})

# Load Pretrained BERT Model and Tokenizer

This section of code loads a pretrained **BERT (Bidirectional Encoder Representations from Transformers)** model and its tokenizer using Hugging Face's `transformers` library.


In [6]:
# Load model directly
model_path = "google-bert/bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_path)

id2label = {0: "Safe", 1: "Not Safe"}
label2id = {"Safe": 0, "Not Safe": 1}
model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                           num_labels=2,
                                                           id2label=id2label,
                                                           label2id=label2id,)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Print Model Layers and Trainable Parameters

This code iterates through all the parameters of the model and prints their names along with their `requires_grad` status.



In [7]:
# print layers
for name, param in model.named_parameters():
   print(name, param.requires_grad)

bert.embeddings.word_embeddings.weight True
bert.embeddings.position_embeddings.weight True
bert.embeddings.token_type_embeddings.weight True
bert.embeddings.LayerNorm.weight True
bert.embeddings.LayerNorm.bias True
bert.encoder.layer.0.attention.self.query.weight True
bert.encoder.layer.0.attention.self.query.bias True
bert.encoder.layer.0.attention.self.key.weight True
bert.encoder.layer.0.attention.self.key.bias True
bert.encoder.layer.0.attention.self.value.weight True
bert.encoder.layer.0.attention.self.value.bias True
bert.encoder.layer.0.attention.output.dense.weight True
bert.encoder.layer.0.attention.output.dense.bias True
bert.encoder.layer.0.attention.output.LayerNorm.weight True
bert.encoder.layer.0.attention.output.LayerNorm.bias True
bert.encoder.layer.0.intermediate.dense.weight True
bert.encoder.layer.0.intermediate.dense.bias True
bert.encoder.layer.0.output.dense.weight True
bert.encoder.layer.0.output.dense.bias True
bert.encoder.layer.0.output.LayerNorm.weight True


# Freezing and Unfreezing Model Parameters

This code is used to selectively freeze and unfreeze the parameters of a pre-trained BERT model during fine-tuning.




In [8]:
# freeze base model parameters
for name, param in model.base_model.named_parameters():
    param.requires_grad = False

# unfreeze base model pooling layers
for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad = True

## Explanation of Printing Model Layers and Trainable Status

This code prints the names of all the parameters in the model along with their `requires_grad` status to verify which layers are trainable and which are frozen.


In [9]:
# print layers
for name, param in model.named_parameters():
   print(name, param.requires_grad)

bert.embeddings.word_embeddings.weight False
bert.embeddings.position_embeddings.weight False
bert.embeddings.token_type_embeddings.weight False
bert.embeddings.LayerNorm.weight False
bert.embeddings.LayerNorm.bias False
bert.encoder.layer.0.attention.self.query.weight False
bert.encoder.layer.0.attention.self.query.bias False
bert.encoder.layer.0.attention.self.key.weight False
bert.encoder.layer.0.attention.self.key.bias False
bert.encoder.layer.0.attention.self.value.weight False
bert.encoder.layer.0.attention.self.value.bias False
bert.encoder.layer.0.attention.output.dense.weight False
bert.encoder.layer.0.attention.output.dense.bias False
bert.encoder.layer.0.attention.output.LayerNorm.weight False
bert.encoder.layer.0.attention.output.LayerNorm.bias False
bert.encoder.layer.0.intermediate.dense.weight False
bert.encoder.layer.0.intermediate.dense.bias False
bert.encoder.layer.0.output.dense.weight False
bert.encoder.layer.0.output.dense.bias False
bert.encoder.layer.0.output.Lay

# Text Preprocessing Function

This cell defines a simple preprocessing function using the tokenizer to prepare text data for input into the model.



In [10]:
# define text preprocessing
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

# Tokenizing All Datasets

This cell applies the previously defined `preprocess_function` to tokenize all the datasets using the `map` function from the Hugging Face Datasets library.



In [11]:
# tokenize all datasetse
tokenized_data = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

# Creating a Data Collator

This cell creates a **Data Collator** using the `DataCollatorWithPadding` class from Hugging Face Transformers.



In [12]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Loading Metrics and Computing Evaluation Metrics

This cell is responsible for evaluating the model's performance using accuracy and AUC (Area Under the Curve) metrics.



In [13]:
# load metrics
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    # get predictions
    predictions, labels = eval_pred

    # apply softmax to get probabilities
    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims=True)
    # use probabilities of the positive class for ROC AUC
    positive_class_probs = probabilities[:, 1]
    # compute auc
    auc = np.round(auc_score.compute(prediction_scores=positive_class_probs, references=labels)['roc_auc'],3)

    # predict most probable class
    predicted_classes = np.argmax(predictions, axis=1)
    # compute accuracy
    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'],3)

    return {"Accuracy": acc, "AUC": auc}

# Explanation of Hyperparameters and TrainingArguments

This cell defines the hyperparameters for the model training process and sets up the training configuration using the `TrainingArguments` class from the Hugging Face Transformers library.


In [14]:
# hyperparameters
lr = 2e-4
batch_size = 8
num_epochs = 10

training_args = TrainingArguments(
    output_dir="bert-phishing-classifier_teacher",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Emissions Tracking and Model Training with CodeCarbon

This cell performs two key functions:  
1. **Training the Model**: It uses the Hugging Face `Trainer` class to fine-tune the BERT model on the phishing site classification dataset.  
2. **Tracking Emissions**: The `EmissionsTracker` from CodeCarbon monitors and reports the carbon footprint generated during the training process.


In [16]:
from codecarbon import EmissionsTracker
import pandas as pd
from transformers import Trainer

# Assuming trainer is already set up
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train the model with emissions tracking
with EmissionsTracker(output_dir='./', output_file='emissions.csv', allow_multiple_runs=True) as tracker:
    trainer.train()

# Read the emissions data from the CSV file
emissions_df = pd.read_csv('emissions.csv')
latest_emissions = emissions_df['emissions'].iloc[-1]
print(f"Total emissions: {latest_emissions} kgCO2eq")

<ipython-input-16-1a76bebd0920>:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
[codecarbon ERROR @ 18:48:44] Error: Another instance of codecarbon is probably running as we find `/tmp/.codecarbon.lock`. Turn off the other instance to be able to run this one or use `allow_multiple_runs` or delete the file. Exiting.
[codecarbon WARNING @ 18:48:44] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 18:48:44] [setup] RAM Tracking...
[codecarbon INFO @ 18:48:44] [setup] CPU Tracking...
[codecarbon WARNING @ 18:48:44] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon WARNING @ 18:48:45] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 18:48:45] CPU Model on

Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.332000,0.275443,0.873000,0.954000
2,0.324800,0.280714,0.887000,0.953000
3,0.297000,0.284749,0.882000,0.953000
4,0.313500,0.323642,0.876000,0.954000
5,0.313700,0.293310,0.882000,0.956000
6,0.315500,0.271854,0.882000,0.956000
7,0.308600,0.273879,0.871000,0.955000
8,0.293200,0.275373,0.873000,0.955000
9,0.295600,0.270155,0.876000,0.956000
10,0.301300,0.276364,0.873000,0.956000


[codecarbon INFO @ 18:49:00] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 18:49:00] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:49:00] Energy consumed for all GPUs : 0.000253 kWh. Total GPU Power : 60.57242975746068 W
[codecarbon INFO @ 18:49:00] 0.000450 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:49:17] Energy consumed for RAM : 0.000043 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 18:49:17] Energy consumed for all CPUs : 0.000380 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:49:17] Energy consumed for all GPUs : 0.000528 kWh. Total GPU Power : 57.611011541919915 W
[codecarbon INFO @ 18:49:17] 0.000950 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:49:32] Energy consumed for RAM : 0.000062 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 18:49:32] Energy consumed for all CPUs : 0.000557 kWh. Total CPU Power : 42.5 W
[codec

Total emissions: 0.0016795223142456 kgCO2eq


/usr/local/lib/python3.11/dist-packages/codecarbon/output_methods/file.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(total.values)])])


# Validation and Emissions Evaluation

This cell performs the following tasks:  
1. **Model Validation**: Applies the trained BERT model to the validation dataset and evaluates its performance.  
2. **Metrics Calculation**: Computes accuracy and AUC using the `compute_metrics()` function.  
3. **Emissions Reporting**: Reads and prints the total carbon emissions generated during training using the data stored in the `emissions.csv` file.  


In [17]:
import pandas as pd

# Apply model to validation dataset
predictions = trainer.predict(tokenized_data["validation"])

# Extract the logits and labels from the predictions object
logits = predictions.predictions
labels = predictions.label_ids

# Use your compute_metrics function
metrics = compute_metrics((logits, labels))

# Read the total emissions from the CSV file generated during training
emissions_df = pd.read_csv('emissions.csv')
total_emissions = emissions_df['emissions'].iloc[-1]

# Print the validation metrics and total emissions together
print("Validation Metrics and Total Emissions:")
print(f"Metrics: {metrics}")
print(f"Total emissions: {total_emissions} kgCO2eq")

Validation Metrics and Total Emissions:
Metrics: {'Accuracy': np.float64(0.902), 'AUC': np.float64(0.949)}
Total emissions: 0.0016795223142456 kgCO2eq
